In [1]:
using LinearAlgebra

# Linear Algebra Reference/Demo
The purpose of this guide is to introduce some of the key ideas of linear algebra, express important theorems and definitions in LaTex, and use Julia to solve important problems in linear algebra as well as visualize certain concepts.

## Table of Contents
<ol> 1.) Linear Systems of Equations, Vectors, and Matrices </ol>
<ol> 2.) Gaussian Elimination and Solving Linear Systems </ol>
<ol> 3.) Vector Spaces </ol>
<ol> 4.) Orthogonality and Determinants </ol>
<ol> 5.) Eigenvalues and Eigenvectors </ol>
<ol> 6.) Singular Value Decomposition, Positive Definite Matrices, Positive Semidefinite Matrices </ol>

## Linear Systems of Equations, Vectors, and Matrices
Before we begin our forray into linear algebra, we need to know what a matrix, vector, and linear system of equations is. We start with the vector, which is simply a collection of numbers. In symbols, we write that a vector **v** with n elements (often called an n-vector) as
$$ \textbf{v} \in \mathbb{R}^n \text{ or } \textbf{v} \in \mathbb{C}^n $$
For example, we can express a 3 vector as 
$$ \textbf{v} = \begin{bmatrix} v_1 \\ v_2 \\ v_3 \end{bmatrix} \text{ or } \begin{bmatrix} v_1 && v_2 && v_3 \end{bmatrix}  $$
Where each element can be a real number or complex number. Note that a 3-vector **v** can be represented by either a column vector or row vector. Vectors are often written with lowercase bold letters, which is the same convention I will try to adopt for this document. <br>
Matrices, like vectors, are also collections of numbers, but they are organized into a rectangular table with m rows and n columns. Matrices can also be viewed as collections of vectors, and a vector can be viewed as a matrix with one row or one column. We express a 2x2 matrix **A** as
$$ \textbf{A} = \begin{bmatrix} a_1 && a_2 \\ a_3 && a_4 \end{bmatrix} = \begin{bmatrix} \textbf{a}_1 && \textbf{a}_2 \end{bmatrix} = \begin{bmatrix} \textbf{a}_1^T \\ \textbf{a}_2^T \end{bmatrix} $$
Note that we use bold uppercase letters to represent matrices, the bold a's are column vectors, and the non-bold a's are scalar values (which can be complex valued for many important matrices). We write 
$$ \textbf{A} \in \mathbb{R}^{mxn} \text{ or } \textbf{A} \in \mathbb{C}^{mxn} $$
to denote that A belongs to the collection of real or complex matrices with m rows and n columns. <br>
Finally, I would like to introduce the concept of a linear system of equations. A system of equations contains m equations where each equation has up to n variables. For example, a 2x2 system of equations can be written as 
$$ \begin{align*}
  a_{11}x_1+a_{12}x_2+a_{13}x_3 &= \, b_1  \\ 
  a_{21}x_1+a_{22}x_2+a_{23}x_3 &= \, b_2\\ 
  a_{31}x_1+a_{32}x_2+a_{33}x_3 &= \, b_3
\end{align*} $$
Alternatively, we can write 
$$ \textbf{Ax} = \textbf{b}$$
where 
$$ A = \begin{bmatrix} a_1 && a_2 && a_3\\ a_4 && a_5 && a_6 \\ a_7 && a_8 && a_9 \end{bmatrix} \text{, } \textbf{x} = \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} \text{, and } \textbf{b} = \begin{bmatrix} b_1 \\ b_2 \\ b_3 \end{bmatrix} $$
Thus, we see that matrices and vectors are convenient structures for solving and manipulating linear systems of equations. Linear systems of equations, vectors, and matrices obey the principle of superposition. When a system or object obeys the principle of superposition, that system or object has the property of linearity, which is defined as
$$ f(\alpha x + \beta y) = \alpha f(x) + \beta f(y) \text{ where } f : \mathbb{R}^n \rightarrow \mathbb{R}^m \text{ and } x,y \in \mathbb{R}^n $$
This notation is very compact, so it is important to clarify in words what is going on here. The notation says that a function is linear iff function inputs can be scaled and added before being transformed or function inputs can be transformed, then scaled, and then added and that performing those operations in either order will yield an equivalent result. 

In [2]:
# How to create a vector in julia
x = [1 2 3]
@show x

# How to create a matrix in julia
# Semicolons mark the end of the row, matrices are represented
# in row major order :) 
A = [1 2 3; 6 5 4; 12 3 2]
@show A

# Matrix transpose ' operator and transpose function 
A = A'
A = transpose(A)
# Transpose of the transpose is itself :)
@show A

# Matrix inversion use inv() for square matrices and pinv() for rectangular
A_inv = inv(A)
@show A_inv
A_rec = [12 3 13; -3 78 24]
A_pinv = pinv(A_rec)
@show A_pinv

#



x = [1 2 3]
A = [1 2 3; 6 5 4; 12 3 2]
A = [1 2 3; 6 5 4; 12 3 2]
A_inv = [0.03571428571428571 -0.08928571428571427 0.12499999999999999; -0.6428571428571428 0.6071428571428571 -0.25; 0.75 -0.375 0.125]
A_pinv = [0.043213703255095326 -0.003754534212040574; -0.010476771799982828 0.012497099058028378; 0.03945122125683112 0.0005817779515693752]


3×2 Array{Float64,2}:
  0.0432137  -0.00375453
 -0.0104768   0.0124971
  0.0394512   0.000581778

## Gaussian Elimination and Solving Linear Systems 
Linear systems of equations are extremely relevant since they are a very natural way to model lots of different phenomanae. In particular, we are often interested in performing estimation/inversion or control/design. Put more abstractly, for a linear system that can be represented as **Ax** = **b** , we want to know what values **x** can give a desired **b** or whether **b** is a value that is reachable given the matrix **A**. In order to answer questions of this nature, we must be able to solve these systems of equations to put them in forms where we can answer these questions and provide exact values for specific systems. 

Here are a couple examples of applications of solving linear systems of equations. As an EE major, I am biased towards circuit applications and will use Julia to solve for the currents and voltages in linear circuits.

![](linear-circuit-1.jpg)


In [3]:
R = [1 -1 0 -1 0; 0 1 -1 0 -1; -1 0 0 -2 0; 0 -1 0 2 -2; 0 0 -1 0 2];
v = [0; 0; -1; 0; 2];
i = R \ v;
print("i is : ", i)

i is : [0.14285714285714296, -0.2857142857142856, -0.857142857142857, 0.42857142857142855, 0.5714285714285714]

Here is a more applied circuit, namely a wheatstone bridge :)

![](wheatstone-bridge.jpg)




## Vector Spaces

## Orthogonality and Determinants

When we are dealing with vectors and matrices, there are times where we would like to know if a set of vectors goes in different directions or how a matrix encoding a transformation changes the orientation and magnitude caused by the transformation. Like many things in math (although notable exceptions exist), I find that the geometric interpretations of orthoganlity and determinants to give me the best intuition of what those concepts are and why they are useful to know when using linear algebra in a practical setting. 

Here is a photo that is meant to represent orthogonality in 2 dimension and 3 dimensions, although this idea extends to n dimensions. However, once we start extending orthogonality to n dimensions, it is often times more convenient to define orthogonality using an inner product and vector notation. Without further ado, here is the artistic masterpiece the masses have been waiting for 

![](orthogonality.jpg)

Note that orthogonality has to do with direction, while determinant encompases both direction and magnitude. Notice how all of the vectors can be multiplied by an scalar value and while remaining orthogonal with each other.
At last, we arrive at the definition of orthogonality. We say two vectors **x** and **y** are orthogonal iff
$$ x^Ty = 0 \text{ for } x \text{, } y \in \mathbb{R}^n $$

## Eigenvalues and Eigenvectors

## Singular Value Decomposition